In [1]:
using ParallelTemperingMonteCarlo
using Profile,JET,Random,DelimitedFiles,BenchmarkTools


Major problems with the RuNNerpotential due to the symmetryfunction vector being type non-specific

In [2]:
Random.seed!(1234)
n_atoms = 55
ti = 750.
tf = 800.
n_traj = 4
nmtobohr = 18.8973
temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details

mc_cycles = 100  #default 20% equilibration cycles on top


mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)


ensemble = NVT(n_atoms)
move_strat = MoveStrategy(ensemble)
#-------------------------------------------------------------#
#-----------------------Starting Config-----------------------#
#-------------------------------------------------------------#

ico_55 = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]
#convert to Bohr

copperconstant = 0.36258*nmtobohr
pos_cu55 = copperconstant*ico_55

AtoBohr = 1.8897259886

length(pos_cu55) == n_atoms || error("number of atoms and positions not the same - check starting config")


#histogram information
n_bin = 100

#boundary conditions
bc_cu55 = SphericalBC(radius=14*AtoBohr)   #5.32 Angstrom
start_config = Config(pos_cu55, bc_cu55)

Config{55, SphericalBC{Float64}, Float64}(StaticArraysCore.SVector{3, Float64}[[4.5112139495856e-6, -1.3138293967695e-5, 3.4601504321700004e-7], [-3.980885942754e-6, -3.3375035158614e-6, 4.575916827038472], [1.264752985534391, -3.8925157884546686, 2.0464230925690106], [-3.311163519470279, -2.405710469814843, 2.0464114164455425], [-3.311163596895427, 2.40569285113995, 2.0464011106786812], [1.264752859461583, 3.8924983184634674, 2.046406418069819], [4.092824837722801, -8.6887460654154e-6, 2.046420003785219], [-1.2647438377188622, -3.8925245957365804, -2.046405726039733], [-4.092815815294902, -1.75878418699746e-5, -2.0464193117551326], [-1.26474396379167, 3.8924895111815556, -2.046422400538924]  …  [2.0744963372369405, 6.384583971948036, -4.148977939714466], [6.5594487104586, 4.765688045720408, -4.053952548518178], [6.713184197946128, -2.1916113212552402e-5, -4.1489556556605045], [6.559448864623718, -4.765731476432347, -4.053932133630629], [-1.2820847296802604, -3.9459310282596545, -6.713

In [3]:
script_folder = @__DIR__ # folder where this script is located
data_path = joinpath(script_folder, "data") # path to data files, so "./data/"

X = [ 1    1              0.001   0.000  11.338
 1    0              0.001   0.000  11.338
 1    1              0.020   0.000  11.338
 1    0              0.020   0.000  11.338
 1    1              0.035   0.000  11.338
 1    0              0.035   0.000  11.338
 1    1              0.100   0.000  11.338
 1    0              0.100   0.000  11.338
 1    1              0.400   0.000  11.338
 1    0              0.400   0.000  11.338]

radsymmvec = []


#--------------------------------------------#
#--------Vector of angular symm values-------#
#--------------------------------------------#
V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.338],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]

T = [[1.,1.,1.],[1.,1.,0.],[1.,0.,0.]]

angularsymmvec = []
#-------------------------------------------#
#-----------Including scaling data----------#
#-------------------------------------------#
file = open(joinpath(data_path,"scaling.data")) # full path "./data/scaling.data"
scalingvalues = readdlm(file)
close(file)
G_value_vec = []
for row in eachrow(scalingvalues[1:88,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec,max_min)
end


for symmindex in eachindex(eachrow(X))
    row = X[symmindex,:]
    radsymm = RadialType2{Float64}(row[3],row[5],[row[1],row[2]],G_value_vec[symmindex])
    push!(radsymmvec,radsymm)
end


let n_index = 10

for element in V
    for types in T

        n_index += 1

        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types,G_value_vec[n_index])

        push!(angularsymmvec,symmfunc)
    end
end
end

Normally we vcat the two vectors. we will try something different here


In [4]:
num_nodes::Vector{Int32} = [88, 20, 20, 1]
activation_functions::Vector{Int32} = [1, 2, 2, 1]
file = open(joinpath(data_path, "weights.029.data"), "r+") # "./data/weights.029.data"
weights=readdlm(file)
close(file)
weights = vec(weights)
nnp = NeuralNetworkPotential(num_nodes,activation_functions,weights)

totalsymmvec = vcat(radsymmvec,angularsymmvec)
runnerpotential = RuNNerPotential(nnp,totalsymmvec)

RuNNerPotential(NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-1.0768943782, 0.3563458393, -1.3084861447, 0.1127640916, -0.5646591931, -0.4969830793, 1.423770514, -0.2005662393, -0.8957859374, 0.6076110858  …  0.1939524848, -0.0342903191, -0.0231042009, 0.0718481968, -0.0828320122, -0.0728501885, 0.0306628826, 0.0486053813, -0.0867018862, 0.0441651841]), AbstractSymmFunction[RadialType2{Float64}(0.001, 11.338, [1.0, 1.0], -0.029892621990594427, 0.06634475788973929), RadialType2{Float64}(0.001, 11.338, [1.0, 0.0], -0.0, 0.09828103647521905), RadialType2{Float64}(0.02, 11.338, [1.0, 1.0], -0.013430592462258787, 0.13321466604629645), RadialType2{Float64}(0.02, 11.338, [1.0, 0.0], -0.0, 0.18618835781674212), RadialType2{Float64}(0.035, 11.338, [1.0, 1.0], -0.0067056953127401205, 0.2093521117141925), RadialType2{Float64}(0.035, 11.338, [1.0, 0.0], -0.0, 0.28571045376567983), RadialType2{Float64}(0.1, 11.338, [1.0, 1.0], -0.00026435144512647774, 0.9364998977117353

In [6]:

@benchmark ptmc_run!(mc_params,temp,start_config,runnerpotential,ensemble)

params set
equilibration complete
MC loop done.
testing revise pt2
[72.23744728735385, 115.53714583271261, 81.32700729270748, 128.4112091753097]
done
params set
equilibration complete
MC loop done.
testing revise pt2
[88.55012696480131, 81.290293258144, 129.35578807435311, 125.14782079593013]
done
params set
equilibration complete
MC loop done.
testing revise pt2
[49.5980927394434, 90.64005227647571, 95.25790339860161, 171.73027058199105]
done


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 251.024 s (4.34% GC) to evaluate,
 with a memory estimate of 222.27 GiB, over 2959501455 allocations.

In [5]:
struct TestPotential{Nrad,Nang} <: AbstractMachineLearningPotential
    nnp:: NeuralNetworkPotential
    radsymfunctions::Vector{RadialType2}
    angsymfunctions::Vector{AngularType3}
    r_cut::Float64
end
function TestPotential(nnp,radsymmvec,angsymmvec)
    r_cut= radsymmvec[1].r_cut 
    nrad = length(radsymmvec)
    nang = length(angsymmvec)
    return TestPotential{nrad,nang}(nnp,radsymmvec,angsymmvec,r_cut)
end


symmsize (generic function with 1 method)

First work on eliminating the run-time dispatch error, then reduce allocs

In [12]:
function total_symm_calc(positions,dist2_mat,f_mat,pot::TestPotential{Nrad,Nang}) where {Nrad,Nang}

    g_mat = zeros(sum(lengthsvec),length(positions))

    for g_index in 1:Nrad
        g_mat[g_index,:] = calc_symm_vals!(positions,dist2_mat,f_mat,g_mat[g_index,:],pot.radsymmfunctions[g_index])
    end
    for g_index in Nrad+1:Nang
        g_mat[g_index,:] = calc_symm_vals!(positions,dist2_mat,f_mat,g_mat[g_index,:],pot.angsymmfunctions[g_index-Nrad])
    end
    return g_mat 
end
function total_symm!(g_matrix,position,new_position,dist2_matrix,new_dist_vector,f_matrix,new_f_vector,atomindex,pot::TestPotential{Nrad,Nang}) where {Nrad,Nang} 
    for g_index in 1:Nrad
        g_matrix[g_index,:] = symmetry_calculation!(g_matrix[g_index,:],atomindex,new_position,position,dist2_matrix,new_dist_vector,f_matrix,new_f_vector,pot.radsymmfunctions)
    end
    for g_index in 1+Nrad:Nang
        symmetry_calculation!(g_matrix[g_index,:],atomindex,new_position,position,dist2_matrix,new_dist_vector,f_matrix,new_f_vector,pot.angsymmfunctions[g_index-Nrad])
    end
    return g_matrix 
end
function get_new_state_vars(trial_pos,atomindex,config,potential_variables::NNPVariables,dist2_mat,pot::TestPotential)
    new_dist2_vec = [ distance2(trial_pos,b,config.bc) for b in config.pos]
    new_dist2_vec[atomindex] = 0.
    potential_variables.new_f_vec = cutoff_function.(sqrt.(new_dist2_vec),Ref(pot.r_cut))
    potential_variables.new_g_matrix = copy(potential_variables.g_matrix)
    potential_variables.new_g_matrix = total_symm(potential_variables.new_g_matrix,config.pos,trial_pos,dist2_mat,new_dist2_vec,potential_variables.f_matrix,potential_variables.new_f_vec,atomindex,pot)
    return new_dist2_vec,potential_variables
end
function initialise_energy(config,dist2_mat,potential_variables,ensemble_variables,pot::TestPotential)
    potential_variables.en_atom_vec = forward_pass(potential_variables.g_matrix,length(config),pot.nnp)
    en_tot = sum(potential_variables.en_atom_vec)
    return en_tot,potential_variables
end
function calc_new_runner_energy!(potential_variables::NNPVariables,pot::TestPotential)
    potential_variables.new_en_atom = forward_pass(potential_variables.new_g_matrix,length(potential_variables.en_atom_vec),pot.nnp)
    new_en = sum(potential_variables.new_en_atom)
    return potential_variables,new_en
end
function energy_update!(trial_pos,index,config::Config,potential_variables::NNPVariables,dist2_mat,en_tot,pot::TestPotential)

    new_dist2_vec,potential_variables = get_new_state_vars!(trial_pos,index,config,potential_variables,dist2_mat,pot)

    potential_variables,new_en = calc_new_runner_energy!(potential_variables,pot)

    return potential_variables,new_dist2_vec,new_en
end

function set_variables(config::Config{N,BC,T},dist2_mat,pot::TestPotential) where {N,BC,T}
    
    f_matrix = cutoff_function.(sqrt.(dist2_mat),Ref(pot.r_cut))
    g_matrix = total_symm_calc(config.pos,dist2_mat,f_matrix,pot)
    
    return NNPVariables{T}(zeros(N) ,zeros(N),g_matrix,f_matrix,zeros(sum(symmsize(pot))), zeros(N))
end

set_variables (generic function with 1 method)

In [9]:
testpot= TestPotential(nnp,radsymmvec,angularsymmvec)

TestPotential{10, 78}(NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-1.0768943782, 0.3563458393, -1.3084861447, 0.1127640916, -0.5646591931, -0.4969830793, 1.423770514, -0.2005662393, -0.8957859374, 0.6076110858  …  0.1939524848, -0.0342903191, -0.0231042009, 0.0718481968, -0.0828320122, -0.0728501885, 0.0306628826, 0.0486053813, -0.0867018862, 0.0441651841]), RadialType2[RadialType2{Float64}(0.001, 11.338, [1.0, 1.0], -0.029892621990594427, 0.06634475788973929), RadialType2{Float64}(0.001, 11.338, [1.0, 0.0], -0.0, 0.09828103647521905), RadialType2{Float64}(0.02, 11.338, [1.0, 1.0], -0.013430592462258787, 0.13321466604629645), RadialType2{Float64}(0.02, 11.338, [1.0, 0.0], -0.0, 0.18618835781674212), RadialType2{Float64}(0.035, 11.338, [1.0, 1.0], -0.0067056953127401205, 0.2093521117141925), RadialType2{Float64}(0.035, 11.338, [1.0, 0.0], -0.0, 0.28571045376567983), RadialType2{Float64}(0.1, 11.338, [1.0, 1.0], -0.00026435144512647774, 0.9364998977117353), 

In [11]:
ptmc_run!(mc_params,temp,start_config,testpot,ensemble)

MethodError: MethodError: no method matching set_variables(::Config{55, SphericalBC{Float64}, Float64}, ::Matrix{Float64}, ::TestPotential{10, 78})

Closest candidates are:
  set_variables(::Config{N, BC, T}, ::Any, !Matched::RuNNerPotential) where {N, BC, T}
   @ ParallelTemperingMonteCarlo ~/Code/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:752
  set_variables(::Config{N, BC, T}, ::Any, !Matched::EmbeddedAtomPotential) where {N, BC, T}
   @ ParallelTemperingMonteCarlo ~/Code/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:744
  set_variables(::Config{N, BC, T}, ::Matrix, !Matched::AbstractDimerPotentialB) where {N, BC, T}
   @ ParallelTemperingMonteCarlo ~/Code/ParallelTemperingMonteCarlo.jl/src/EnergyEvaluation.jl:739
  ...
